In [1]:
from sklearn import preprocessing
import pandas as pd 
from datetime import datetime
import xgboost as xgb
df = pd.read_hdf('../../data/processed/train/aggr_train_zeros.hdf')

threshold = datetime(2015, 9, 1)

df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-1')

In [2]:
from sklearn import preprocessing
def X(df):
    df['day_of_year'] = df['date'].dt.dayofyear
    date_cols = [
        'month', 'year',
        'fridays', 'day_of_year', 'days_in_a_month'
    ]
    one_hot_cols = [
        'shop_id', 'mall', 'city',
        'item_id', 'item_category_id', 'first_big_category', 'last_big_category',
    ]
    additional = [
        'expected_sales'
    ]

    df_X = df[
        [
            col for col in df.columns
            if col.startswith('count')
        ]
        + date_cols + one_hot_cols + additional
        ]

    # df_X = pd.get_dummies(df_X, columns=one_hot_cols)
    # df_X = pd.get_dummies(df_X, columns=one_hot_cols + date_cols)
    for f in df_X.columns:
        if df_X[f].dtype == 'object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(df_X[f].values))
            df_X[f] = lbl.transform(list(df_X[f].values))
    return df_X

def XY(df):
    df_X = X(df)
    df_y = df['item_cnt_day']
    return df_X, df_y

In [3]:
df_train = df[df['date'] < threshold]
#df_train = df
df_X_train, df_y_train = XY(df_train)

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors = 10)
model.fit(df_X_train, df_y_train.clip(0, 20))

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='uniform')

In [9]:
import pickle 
pickle.dump(model, open('../../models/KNN_10_all.hdf', 'wb'))

MemoryError: 

In [6]:
from sklearn.metrics import r2_score, mean_squared_error
from math import sqrt
def get_preds_trained_on_all(model, df_X):
    return model.predict(df_X).clip(0, 20)


def previous_value_preds(df_X):
    df_X = df_X.reindex(columns=df_X_train.columns, fill_value=0)
    return df_X['count_aggr_1_month_ago_shop_id_item_id'].clip(0, 20)


def validate(model, df_y_valid, df_X_valid):
    df_y_valid = df_y_valid.copy().clip(0, 20)
    df_y_valid_preds = get_preds_trained_on_all(model, df_X_valid)
    # df_y_valid_preds = previous_value_preds(df_X_valid)
    print('r2 score: ', r2_score(df_y_valid, df_y_valid_preds))
    print('RMSE: ', sqrt(mean_squared_error(df_y_valid, df_y_valid_preds)))

In [7]:
df_valid = df[df['date'] >= threshold]

df_X_valid, df_y_valid = XY(df_valid)
validate(model, df_y_valid, df_X_valid)

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


r2 score:  0.20257526685991722
RMSE:  0.984892416406698


In [16]:
df_X_valid['preds'] = get_preds_trained_on_all(model, df_X_valid)
a = df_X_valid['preds'].to_hdf('../../data/processed/model_valid/KNN_10.hdf', 'KNN_10')

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df_X_valid['preds'] = get_preds_trained_on_all(model, df_X_valid)
a = df_X_valid['preds'].to_hdf('../../data/processed/model_valid/pre_valid_train_clip/KNN_10.hdf', 'KNN_10')

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df_test = pd.read_hdf('../../data/processed/test/aggr_test.hdf')
df_test_X = X(df_test)
df_test['linreg_preds'] = get_preds_trained_on_all(model, df_test_X)
df_test[['ID', 'linreg_preds']].to_hdf('../../data/processed/model_valid/knn10_test.hdf', 'knn10_test')

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
df_test = pd.read_hdf('../../data/processed/test/aggr_test.hdf')
df_test_X = X(df_test)
df_test['linreg_preds'] = get_preds_trained_on_all(model, df_test_X)
df_test[['ID', 'linreg_preds']].to_hdf('../../data/processed/model_valid/all_train_clip/knn10_test.hdf', 'knn10_test')

/home/piotrek/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
